In [129]:
import networkx as nx
import numpy as np
from heapq import heappush, heappop
import time
from memory_profiler import memory_usage
import tsplib95

### TWICE AROUND THE TREE

In [130]:
def twiceAroundTheTreeTSP(A):
    MST = nx.minimum_spanning_tree(A)
    caminho = list(nx.dfs_preorder_nodes(MST, 0))
    caminho.append(caminho[0])
    peso = sum(A[caminho[i]][caminho[i + 1]]['weight'] for i in range(len(caminho) - 1))

    return peso, caminho


### CHRISTOFIDES

In [131]:
def encontrarCaminhoCurto(A):
    caminho = [x[0] for x in nx.eulerian_circuit(A, 0)]
    return list(dict.fromkeys(caminho)) + [caminho[0]]

def christofidesTSP(A):
    MST = nx.minimum_spanning_tree(A)
    nosImpares = [no for no, grau in nx.degree(MST) if grau % 2 == 1]
    pareamento = nx.min_weight_matching(nx.subgraph(A, nosImpares))

    MSTMultiGrafo = nx.MultiGraph(MST)
    MSTMultiGrafo.add_edges_from((no1, no2, A[no1][no2]) for no1, no2 in pareamento)

    caminho = encontrarCaminhoCurto(MSTMultiGrafo)
    peso = sum(A[caminho[i]][caminho[i + 1]]['weight'] for i in range(len(caminho) - 1))

    return peso, caminho

#### BRANCH AND BOUND

In [132]:
import numpy as np

class No:
    def __init__(self, limite, arestasLimite, custo, solucao):
        self.limite = limite
        self.arestasLimite = arestasLimite
        self.custo = custo
        self.solucao = solucao
    
    def __lt__(self, outro):
        if len(self.solucao) == len(outro.solucao):
            return self.limite < outro.limite
        return len(self.solucao) > len(outro.solucao)

def encontrarDuasArestasMinimas(lista):
    min1, min2 = np.inf, np.inf
    for j in lista:
        peso = lista[j]['weight']
        if peso < min1:
            min1, min2 = peso, min1
        elif peso < min2:
            min2 = peso
    return min1, min2

def encontrarLimiteInicial(A):
    limite = 0
    arestasLimiteIniciais = np.zeros((A.number_of_nodes(), 2), dtype=list)
    for i in range(A.number_of_nodes()):
        min1, min2 = encontrarDuasArestasMinimas(A[i])
        arestasLimiteIniciais[i] = [min1, min2]
        limite += min1 + min2
    return limite / 2, arestasLimiteIniciais

def encontrarLimite(A, solucao, arestasLimite, limite):
    arestasAlteradas = np.zeros(A.number_of_nodes(), dtype=int)
    novasArestas = np.array(arestasLimite)
    pesoAresta = A[solucao[-2]][solucao[-1]]['weight']
    soma = limite * 2

    for no in solucao[-2:]:
        if novasArestas[no][0] != pesoAresta:
            soma -= novasArestas[no][arestasAlteradas[no]]
            soma += pesoAresta
            arestasAlteradas[no] += 1

    return soma / 2, novasArestas


In [133]:
def branchAndBoundTSP(A):
    limiteInicial, arestasLimiteIniciais = encontrarLimiteInicial(A)
    raiz = No(limiteInicial, arestasLimiteIniciais, 0, [0])
    heap = []
    heappush(heap, raiz)
    melhorCusto = np.inf
    melhorSolucao = []
    contadorDeNos = 0

    while heap:
        noAtual = heappop(heap)
        contadorDeNos += 1
        nivel = len(noAtual.solucao)

        if nivel > A.number_of_nodes():
            if melhorCusto > noAtual.custo:
                melhorCusto = noAtual.custo
                melhorSolucao = noAtual.solucao
        else:
            if noAtual.limite < melhorCusto:
                if nivel < A.number_of_nodes() - 2:
                    for k in range(1, A.number_of_nodes()):
                        if k in noAtual.solucao or k == 0:
                            continue
                        pesoAresta = A[noAtual.solucao[-1]][k]['weight']
                        novoLimite, novasArestas = encontrarLimite(A, noAtual.solucao + [k], noAtual.arestasLimite, noAtual.limite)
                        if novoLimite < melhorCusto:
                            novoNo = No(novoLimite, novasArestas, noAtual.custo + pesoAresta, noAtual.solucao + [k])
                            heappush(heap, novoNo)
                else:
                    for k in range(1, A.number_of_nodes()):
                        if k in noAtual.solucao or k == 0:
                            continue
                        ultimoNo = next(i for i in range(1, A.number_of_nodes()) if i not in noAtual.solucao + [k] and k != i)
                        pesoAresta = A[noAtual.solucao[-1]][k]['weight']
                        proxPesoAresta = A[k][ultimoNo]['weight']
                        ultimoPesoAresta = A[ultimoNo][0]['weight']
                        custo = noAtual.custo + pesoAresta + proxPesoAresta + ultimoPesoAresta
                        if custo < melhorCusto:
                            novoNo = No(custo, [], custo, noAtual.solucao + [k, ultimoNo, 0])
                            heappush(heap, novoNo)

    return melhorCusto, melhorSolucao


#### Criando grafos de teste

In [134]:
A = np.array([[0, 3, 1, 5, 8],
    [3, 0, 6, 7, 9],
    [1, 6, 0, 4, 2],
    [5, 7, 4, 0, 3],
    [8, 9, 2, 3, 0]])

A = nx.from_numpy_array(np.matrix(A), create_using=nx.Graph)

B = np.array([[0, 4, 8, 9, 12],
    [4, 0, 6, 8, 9],
    [8, 6, 0, 10, 11],
    [9, 8, 10, 0, 7],   
    [12, 9, 11, 7, 0]])

B = nx.from_numpy_array(np.matrix(B), create_using=nx.Graph)

C = nx.complete_graph(15)
for (u, v, w) in C.edges(data=True):
    w['weight'] = np.random.randint(1, 10)

In [135]:
def executarAlgoritmo(algoritmo, dados):
    tempo_inicio = time.time()
    tempo_maximo = 30 * 60  # 30 minutos em segundos

    try:
        # Executa o algoritmo e mede o uso de memória
        uso_memoria = memory_usage((algoritmo, (dados,)))
        resultado = algoritmo(dados)
        tempo_fim = time.time()

        tempo_execucao = tempo_fim - tempo_inicio
        if tempo_execucao > tempo_maximo:
            raise TimeoutError
        
        print("Algoritmo: ", algoritmo.__name__)
        print("Peso: ", resultado[0])
        print("Caminho: ", resultado[1])
        print("Tempo de execução: ", tempo_execucao)
        print("Uso de memória: ", np.max(uso_memoria))
        print()

        return resultado, tempo_execucao, np.max(uso_memoria)
    except TimeoutError:
        return "NA", "NA", "NA"

executarAlgoritmo(twiceAroundTheTreeTSP, A)
executarAlgoritmo(christofidesTSP, B)
executarAlgoritmo(branchAndBoundTSP, C)


Algoritmo:  twiceAroundTheTreeTSP
Peso:  16
Caminho:  [0, 2, 4, 3, 1, 0]
Tempo de execução:  0.07798051834106445
Uso de memória:  112.58984375

Algoritmo:  christofidesTSP
Peso:  38
Caminho:  [0, 1, 3, 4, 2, 0]
Tempo de execução:  0.06716775894165039
Uso de memória:  112.58984375



Algoritmo:  branchAndBoundTSP
Peso:  33
Caminho:  [0, 4, 9, 6, 13, 10, 7, 5, 2, 11, 12, 14, 8, 1, 3, 0]
Tempo de execução:  0.40019965171813965
Uso de memória:  112.58984375



((33, [0, 4, 9, 6, 13, 10, 7, 5, 2, 11, 12, 14, 8, 1, 3, 0]),
 0.40019965171813965,
 112.58984375)

In [136]:
def carregar_problema_tsp(caminho_arquivo):
    problema = tsplib95.load(caminho_arquivo)
    grafo = problema.get_graph()
    return grafo

def carregar_solucao_optima(caminho_arquivo):
    with open(caminho_arquivo, 'r') as file:
        linhas = file.readlines()
        tour = [int(linha.strip()) for linha in linhas if linha.strip().isdigit()]
    return tour

def calcular_peso_solucao_optima(grafo, tour_optima):
    peso_total = 0
    for i in range(len(tour_optima) - 1):
        peso_total += grafo[tour_optima[i]][tour_optima[i + 1]]['weight']
    # Adiciona o peso para retornar ao ponto de partida
    peso_total += grafo[tour_optima[-1]][tour_optima[0]]['weight']
    return peso_total


grafo = carregar_problema_tsp('datasets/berlin52.tsp')
tour_optima = carregar_solucao_optima('datasets/berlin52.opt.tour')

peso_solucao_optima = calcular_peso_solucao_optima(grafo, tour_optima)

print("Peso da solução ótima:", peso_solucao_optima)
#executarAlgoritmo(twiceAroundTheTreeTSP, grafo)
#executarAlgoritmo(christofidesTSP, grafo)
executarAlgoritmo(branchAndBoundTSP, grafo)


Peso da solução ótima: 7542


KeyError: 0